In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x3894b90
Welcome to JupyROOT 6.30/06


In [2]:
%jsroot on

In [3]:
ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.gInterpreter.Declare("#include <podio/GenericParameters.h>")

True

In [5]:
ROOT.ildStyle.SetOptStat(1)
ROOT.ildStyle.SetPalette(ROOT.kBird)

In [6]:
%%cpp
using namespace ROOT::VecOps;

In [7]:
df = ROOT.RDataFrame("events", (
    "data/gen/sw_sl/eLpL.0.edm4hep.root",
    "data/gen/sw_sl/eLpR.0-6.edm4hep.root",
    "data/gen/sw_sl/eRpR.0.edm4hep.root",
    "data/gen/sw_sl/eRpL.0.edm4hep.root",
    )
)
# df = ROOT.RDataFrame("events", "data/truejet/test/sw_sl/eLpL_truejet.edm4hep.root")
# df = ROOT.RDataFrame("events", "data/truejet/test/sw_sl/eLpR_truejet.edm4hep.root")

In [8]:
# useless in a notebook
# ROOT.RDF.Experimental.AddProgressBar(df)

In [9]:
df.Describe()

Dataframe from TChain events in files
  data/gen/sw_sl/eLpL.0.edm4hep.root
  data/gen/sw_sl/eLpR.0-6.edm4hep.root
  data/gen/sw_sl/eRpR.0.edm4hep.root
  data/gen/sw_sl/eRpL.0.edm4hep.root

Property                Value
--------                -----
Columns in total           67
Columns from defines        0
Event loops run             0
Processing slots            6

Column                                  Type                                            Origin
------                                  ----                                            ------
EventHeader                             ROOT::VecOps::RVec<edm4hep::EventHeaderData>    Dataset
EventHeader.eventNumber                 ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.runNumber                   ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.timeStamp                   ROOT::VecOps::RVec<ULong_t>                     Dataset
EventHeader.weight                      ROOT::VecOps::RV

In [10]:
%%cpp
auto bPol_e = [](unsigned int slot, const ROOT::RDF::RSampleInfo &id) { return id.Contains("eL") ? -1 : 1; };
auto bPol_p = [](unsigned int slot, const ROOT::RDF::RSampleInfo &id) { return id.Contains("pL") ? -1 : 1; };
auto nevents = [](unsigned int slot, const ROOT::RDF::RSampleInfo &id) {
    // manually extracted :(, not possible to load from json yet because root in the stack is too old
    if (id.Contains("eLpL")) {
        return 100000;
    } else if (id.Contains("eLpR")) {
        return 700000;
    } else if (id.Contains("eRpR")) {
        return 100000;
    } else if (id.Contains("eRpL")) {
        return 100000;
    } else {
        return 0; //should not happen...
    }
};

In [11]:
%%cpp
using namespace ROOT::VecOps;

In [12]:
df = df.DefinePerSample("beamPol_e", "bPol_e(rdfslot_, rdfsampleinfo_)")
df = df.DefinePerSample("beamPol_p", "bPol_p(rdfslot_, rdfsampleinfo_)")
df = df.DefinePerSample("n_events", "nevents(rdfslot_, rdfsampleinfo_)")

# df = df.Define("xsec_fb", "PARAMETERS._floatMap.second[PARAMETERS._floatMap.first == crossSection]")
df = df.Define("xsec_fb", "PARAMETERS.getValue<float>(\"crossSection\")")

In [13]:
# what do I want to have?  Leading (stable) e and nu?
df = df.Define("stable_l_mask", "abs(MCParticles.PDG) == 11 && MCParticles.generatorStatus == 1")
df = df.Define("stable_l_lvec", "Construct<ROOT::Math::PxPyPzMVector>(MCParticles.momentum.x[stable_l_mask], MCParticles.momentum.y[stable_l_mask], MCParticles.momentum.z[stable_l_mask], MCParticles.mass[stable_l_mask])")
df = df.Define("leading_l_lvec_idx", "return ArgMax(Map(stable_l_lvec, [](const auto &el) {return el.energy();}));")
df = df.Define("leading_l_lvec", "stable_l_lvec[leading_l_lvec_idx]")

df = df.Define("stable_nu_mask", "abs(MCParticles.PDG) == 12 && MCParticles.generatorStatus == 1")
df = df.Define("stable_nu_lvec", "Construct<ROOT::Math::PxPyPzMVector>(MCParticles.momentum.x[stable_nu_mask], MCParticles.momentum.y[stable_nu_mask], MCParticles.momentum.z[stable_nu_mask], MCParticles.mass[stable_nu_mask])")
df = df.Define("leading_nu_lvec_idx", "return ArgMax(Map(stable_nu_lvec, [](const auto &el) {return el.energy();}));")
df = df.Define("leading_nu_lvec", "stable_nu_lvec[leading_nu_lvec_idx]")

In [14]:
# df.Display(("stable_l_mask", "stable_l_lvec_e", "leading_l_lvec_idx"), 10).Print()

In [15]:
df = df.Alias("l_lvec", "leading_l_lvec")
df = df.Alias("nu_lvec", "leading_nu_lvec")

In [16]:
df = df.Define("l_lvec_e", "l_lvec.energy()")
df = df.Define("nu_lvec_e", "nu_lvec.energy()")

In [17]:
# weight calc
ePol_target = 0.0
pPol_target = 0.0

lumi_target = 500. # fb^{-1}

# polweight = (1 + epol*epol_target)*(1 + ppol*ppol_target)/4.;

# assume one file per process-polarisation combination
# - in case of several files per process-polarisation combination, need to add up *beforehand*
# the numbers of events of all the files belonging to the same process-polarisation combination!
# weight = polweight * xsection * lumi_target / nevt_per_file[ifile];

# nevt_per_pol = {}
# polweight = {}

# def polfilter(ePol: int, pPol: int) -> str:
#     return f"beamPol_e == {ePol} && beamPol_p == {pPol}"

# from itertools import product

# for ePol, pPol in product([-1, +1], repeat=2):
#     # FIXME: needed for more than one file per pol! also needs summing of the crosssections then!
#     # nevt_per_pol[(ePol, pPol)] = df.Filter(polfilter(ePol, pPol)).Count()
#     nevt_per_pol[(ePol, pPol)] = 100000
#     polweight[(ePol, pPol)] = (1 + ePol * ePol_target) * (1 + pPol * pPol_target) / 4

df = df.Define("weight", f"1./4. * xsec_fb * {lumi_target} / (double) n_events")

In [18]:
h_2d_lnu_e = df.Histo2D(("", ";E_{e} [GeV];E_{#nu} [GeV]", 300, 0., 150., 300, 0., 150.), "l_lvec_e", "nu_lvec_e", "weight")

h_n_events = df.Histo1D("n_events")
h_weight = df.Histo1D("weight")
h_xsec_fb = df.Histo1D("xsec_fb")

In [19]:
c_2d_lnu_e = ROOT.TCanvas()
h_2d_lnu_e.Draw("colz0")
c_2d_lnu_e.Draw()

c_n_events = ROOT.TCanvas()
h_n_events.Draw("hist")
c_n_events.Draw()

c_weight = ROOT.TCanvas()
h_weight.Draw("hist")
c_weight.Draw()

c_xsec_fb = ROOT.TCanvas()
h_xsec_fb.Draw("hist")
c_xsec_fb.Draw()

In [20]:
# FIXME: for debug
# multi_qq_evts = df.Filter("l_MC_idx > 20").Take["ULong64_t"]("rdfentry_").GetValue()
# print(multi_qq_evts)

In [21]:
# WARNING: l_MC_lvec + nu_MC_lvec != lnu_ICN_lvec as we used the final elementon for l/nu_MC not the initial!
# df.Snapshot("events", "data/truejet/test/sw_sl/nano.root", ("beamPol_e", "beamPol_p", "xsec_fb", "weight", "l_MC_lvec", "nu_MC_lvec", "l_MC_pdg", "nu_MC_pdg", "lnu_ICN_lvec", "qq_ICN_lvec"))